# Clustering

This notebook creates a clustering model based on the cropped cells generated by crop_cell_segmentation

In [1]:
import os
import sys

import cv2
import keras
import joblib
import numpy as np
from tqdm.contrib.concurrent import process_map
import tensorflow as tf
from sklearn.cluster import KMeans

from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, DenseNet121, MobileNetV2
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model

sys.path.insert(0, "../../packages/python")
from data import utils as data_utils

2025-05-03 08:50:15.275128: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-03 08:50:15.293207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746273015.301700   27225 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746273015.304362   27225 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746273015.310565   27225 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Definitions

In [2]:
sys.path.insert(0, "../../")
from config import MEDIA_PATH, CROPPED_PATH, MODELS_PATH

# Configuration
INPUT_SHAPE = (128, 128, 3)
NUM_CLUSTERS = 15 # Hand picked
METHOD = 'kmeans' # Options are 'hdbscan', 'kmeans' or 'agglomerative'
MODEL = "VGG16"

# Paths
CROPPED_PATH = os.path.join(CROPPED_PATH, 'ina', 'images')
OUTPUT_PATH = os.path.join(MEDIA_PATH, 'ina', 'clustering_images', f"{MODEL}_{METHOD}_v0") 
MODELS_PATH = os.path.join(MODELS_PATH, 'clustering')

### Functions

In [3]:
def load_image (x):
    if MODEL == "AutoEncoder":
        return cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    else:
        return cv2.imread(x)

### Encoder definition

In [4]:
if MODEL == "AutoEncoder":
    ENCODER_PATH = os.path.join(MODELS_PATH, 'model_Encoder_SSIM+MAE0 (1).keras')
    encoder = keras.saving.load_model(ENCODER_PATH)
else:
    # Load pre-trained models
    if MODEL == 'VGG16':
        preprocess_input = keras.applications.vgg16.preprocess_input
        encoder = VGG16(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    elif MODEL == 'VGG19':
        preprocess_input = keras.applications.vgg19.preprocess_input
        VGG19(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    elif MODEL == 'ResNet50':
        preprocess_input = tf.keras.layers.Identity
        ResNet50(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    elif MODEL == 'InceptionV3':
        preprocess_input = keras.applications.inception_v3.preprocess_input
        InceptionV3(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    elif MODEL == 'DenseNet121':
        preprocess_input = tf.keras.layers.Identity
        DenseNet121(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    elif MODEL == 'MobileNetV2':
        preprocess_input = tf.keras.layers.Identity
        MobileNetV2(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)

    inp = keras.Input(shape=INPUT_SHAPE)
    x = preprocess_input(inp)
    x = encoder(x)
    x = Flatten()(x)
    encoder = Model(inputs=inp, outputs=x)

I0000 00:00:1746273016.789236   27225 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21087 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
encoder.output_shape

(None, 8192)

# Load the images

In [6]:
CROPPED_PATHs = sorted(data_utils.get_relative_file_paths(CROPPED_PATH))

images = process_map(
                load_image,
                CROPPED_PATHs,
                total=len(CROPPED_PATHs),
                max_workers=16,
                chunksize=32,
            )

100%|██████████| 53633/53633 [00:02<00:00, 23386.27it/s]


### Generate encoder embeddings

In [7]:
#Transform input images for encoder input
resized_images = [cv2.resize(image, INPUT_SHAPE[0:2]) for image in images]
resized_images = np.array(resized_images)

# images = images/255 # Esto esta incluido en el modelo

#resized_images = [np.expand_dims(image, axis=(0, -1)) for image in resized_images]

In [8]:
if MODEL == "AutoEncoder":
    resized_images = resized_images / 255.0
    resized_images = np.expand_dims(resized_images, axis=-1)
resized_images.shape

(53633, 128, 128, 3)

In [9]:
encoder.output_shape

(None, 8192)

In [10]:
# Extract features from encoder


enc_features_array = np.zeros((resized_images.shape[0], encoder.output_shape[-1]))

batch_size = 256
ini = 0
while True:
    start = ini*batch_size
    end = start+batch_size

    if start >= resized_images.shape[0]:
        break

    if end >= resized_images.shape[0]:
        end = resized_images.shape[0]-1


    this_batch = resized_images[start:end]

    enc_features_array[start:end] = encoder.predict(this_batch, verbose=0)

    ini += 1



I0000 00:00:1746273021.560243   28015 service.cc:152] XLA service 0x7d3af40043d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746273021.560256   28015 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-05-03 08:50:21.570233: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746273021.613892   28015 cuda_dnn.cc:529] Loaded cuDNN version 90701
I0000 00:00:1746273022.881932   28015 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [11]:
# enc_features_array_norm = [a / (np.linalg.norm(a) + 1e-16) for a in enc_features_array]
enc_features_array_norm = enc_features_array


### Train clustering

In [12]:
if METHOD == 'kmeans':
    kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
    clustering = kmeans.fit(enc_features_array_norm)

    centroids = kmeans.cluster_centers_
    seleccted_class = -np.ones((len(enc_features_array_norm)), dtype=int)
    for idx, feature in enumerate(enc_features_array_norm):
        dist = 1e99
        for cluster in range(NUM_CLUSTERS):
            t_dist = np.linalg.norm(feature - centroids[cluster])
            if dist > t_dist:
                dist = t_dist
                seleccted_class[idx] = cluster




In [13]:
if METHOD == 'agglomerative':
    from sklearn.cluster import AgglomerativeClustering

    # Aca si o si normalizamos
    enc_features_array_norm = [a / (np.linalg.norm(a) + 1e-16) for a in enc_features_array]

    # ag_clustering = AgglomerativeClustering
    #     n_clusters = None,
    #     metric = 'euclidean',
    #     linkage = 'ward',
    #     distance_threshold = 1.0,
    #     compute_full_tree = True,
    # )

    ag_clustering = AgglomerativeClustering(
        n_clusters = NUM_CLUSTERS,
        linkage = 'ward',
    )

    clustering = ag_clustering.fit(enc_features_array_norm)
    seleccted_class = clustering.labels_

In [14]:
if METHOD == 'hdbscan':
    from sklearn.cluster import HDBSCAN
    hdb = HDBSCAN()
    clustering = hdb.fit(enc_features_array_norm)
    seleccted_class = clustering.labels_

In [15]:
os.makedirs(MODELS_PATH, exist_ok=True)
joblib.dump(clustering, os.path.join(MODELS_PATH, f'clustering_{MODEL}_{METHOD}_v0.pkl'))

['/home/nicolas/Documentos/UTN/INA/giar_ina_dev/models/clustering/clustering_VGG16_kmeans_v0.pkl']